# Создание датасета для обучения моделей восстановления изображений
Этот Notebook демонстрирует процесс создания подмножества датасета из DocBank (6000 изображений) с применением искажений (bad_print с aging, brightness_contrast, pixelation).

Изображения делятся на train/val (80/20%), сохраняются пары clean/distorted для каждой задачи.

Исходный датасет можно скачать тут - https://doc-analysis.github.io/docbank-page/index.html

## Шаг 1: Импорт библиотек
Импортируем библиотеки для работы с изображениями, путями и эффектами.

In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path
import random
from tqdm import tqdm
from effects import apply_bad_print_effect, apply_aging_effect, apply_brightness_contrast, apply_pixelation

## Шаг 2: Настройка путей
Определяем пути к исходному датасету и выходной директории.

In [ ]:
dataset_path = Path("D:/DocBank_500K_ori_img")
output_path = Path("D:/DocBank_Subset_6000")
output_path.mkdir(exist_ok=True)

## Шаг 3: Выбор случайных изображений
Выбираем 6000 случайных JPG-файлов из датасета.

In [ ]:
image_files = list(dataset_path.glob("*.jpg"))
random.shuffle(image_files)
selected_files = image_files[:6000]

## Шаг 4: Разделение на задачи
Делим изображения на 3 задачи по 2000 штук (bad_print, brightness_contrast, pixelation).

In [ ]:
images_per_task = 6000 // 3
tasks = {
    "bad_print": selected_files[:images_per_task],
    "brightness_contrast": selected_files[images_per_task:2*images_per_task],
    "pixelation": selected_files[2*images_per_task:]
}

## Шаг 5: Обработка изображений
Для каждой задачи создаём подпапки train/val, применяем эффекты к distorted, сохраняем clean и distorted в RGB.

In [ ]:
for task, files in tasks.items():
    print(f"Обработка задачи: {task}")
    
    train_clean = output_path / task / "train" / "clean"
    train_distorted = output_path / task / "train" / "distorted"
    val_clean = output_path / task / "val" / "clean"
    val_distorted = output_path / task / "val" / "distorted"
    for p in [train_clean, train_distorted, val_clean, val_distorted]:
        p.mkdir(parents=True, exist_ok=True)
    
    train_size = int(0.8 * len(files))
    train_files = files[:train_size]
    val_files = files[train_size:]
    
    for img_path in tqdm(train_files, desc=f"{task} - Train"):
        img = cv2.imread(str(img_path))
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        clean_path = train_clean / img_path.name
        cv2.imwrite(str(clean_path), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        
        if task == "bad_print":
            distorted = apply_bad_print_effect(img)
            distorted = apply_aging_effect(distorted)
        elif task == "brightness_contrast":
            distorted = apply_brightness_contrast(img)
        else:
            distorted = apply_pixelation(img)
        
        distorted_path = train_distorted / img_path.name
        cv2.imwrite(str(distorted_path), cv2.cvtColor(distorted, cv2.COLOR_RGB2BGR))
    
    for img_path in tqdm(val_files, desc=f"{task} - Val"):
        img = cv2.imread(str(img_path))
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        clean_path = val_clean / img_path.name
        cv2.imwrite(str(clean_path), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        
        if task == "bad_print":
            distorted = apply_bad_print_effect(img)
            distorted = apply_aging_effect(distorted)
        elif task == "brightness_contrast":
            distorted = apply_brightness_contrast(img)
        else:
            distorted = apply_pixelation(img)
        
        distorted_path = val_distorted / img_path.name
        cv2.imwrite(str(distorted_path), cv2.cvtColor(distorted, cv2.COLOR_RGB2BGR))